In [ ]:
%reload_ext autoreload
%autoreload 2

import numpy as np
from scipy.interpolate import griddata
import geopandas as gpd
import pandas as pd
import holoviews as hv
hv.extension('bokeh')

import prepare_data
from settings import Config, target

In [ ]:
# What happened so far: DB extract and blank procedure. Now import resulting MP data from csv
mp_pdd = prepare_data.get_pdd()

# Also import sediment data (sediment frequencies per size bin from master sizer export)
grainsize_iow, grainsize_cau, sed_lower_boundaries = prepare_data.get_grainsizes()

# ...some data wrangling to prepare particle domain data and sample domain data for MP and combine with certain sediment aggregates.
mp_sdd = prepare_data.aggregate_SDD(mp_pdd)
sdd_iow = prepare_data.additional_sdd_merging(mp_sdd)
sdd_cau = pd.read_csv('../data/Metadata_CAU_sampling_log.csv', index_col=0)

In [ ]:
all_stations = pd.concat([sdd_iow[['Sample', 'LON', 'LAT', 'Depth']],sdd_cau.reset_index().drop('Date', axis=1)])
# gdf = gpd.GeoDataFrame(all_stations, geometry=gpd.points_from_xy(all_stations['LON'], all_stations['LAT'], crs='EPSG:4326')).to_crs("EPSG:3857")

In [ ]:

gdf = gpd.GeoDataFrame(sdd_iow, geometry=gpd.points_from_xy(sdd_iow['LON'], sdd_iow['LAT'], crs='EPSG:4326')).to_crs("EPSG:3857")

In [ ]:
# create geodataframe from geojson file
poly = gpd.read_file('../data/SchleiCoastline_from_OSM.geojson')

In [ ]:
poly.plot()

In [ ]:
gdf.to_crs(Config.baw_epsg, inplace=True)
poly.to_crs(Config.baw_epsg, inplace=True)

xres = yres = Config.interpolation_resolution
xmin, ymin, xmax, ymax = poly.total_bounds
xgrid, ygrid = np.meshgrid(np.arange(xmin, xmax + xres, xres), 
                           np.arange(ymin, ymax + yres, yres),
                          )

points = np.vstack((gdf.geometry.x, gdf.geometry.y)).T
values = griddata(
    points, gdf[target],
    (xgrid, ygrid),
    method=Config.interpolation_method,  # 'linear' and 'cubic' will result in nan outside of the convex hull of data points
)

# if nonan:
nan_mask = np.isnan(values)  # if there are any nan points re-interpolate them using method 'nearest'
if np.any(nan_mask):
    values2 = griddata(
        points, gdf[target],
        (xgrid, ygrid), method='nearest',
    )
    values[nan_mask] = values2[nan_mask]

clipped = gpd.GeoDataFrame({'value': values.ravel()}, 
                            geometry=gpd.points_from_xy(xgrid.ravel(), ygrid.ravel()),
                            crs=Config.baw_epsg,
                            )
clipped = gpd.overlay(clipped, poly, how='intersection')
clipped = clipped.loc[clipped.intersects(poly.geometry[0])]

cell_areas = xres * yres
total = (clipped['value'] * cell_areas).sum()

In [ ]:
total

In [ ]:
total / poly.area / 5

In [ ]:
hv.Image(values, bounds=(xmin, ymin, xmax, ymax))